# Init values

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
SEED = 42
dataset_folder = "datasets/financial"
# dataset_folder = "datasets/healthcare"
# dataset_folder = "datasets/eligibility"

MAX_ITERATIONS = 1
TEMPERATURE = 1.0
LLM_NAME = "meta-llama/llama-3.1-8b-instruct:free"
# LLM_NAME = "meta-llama/llama-3.1-8b-instruct:free"


In [3]:
from dotenv import load_dotenv
from langchain_community.cache import SQLiteCache
from langchain.globals import set_llm_cache
from caia.utils import save_yaml_results
from caia.utils import ChatOpenRouter

import json
import uuid


# dataset_folder = "datasets/financial"
with open(f'{dataset_folder}/dataset_description.json', 'r') as f:
    dataset_description = json.load(f)

load_dotenv("env")
# set_llm_cache(SQLiteCache(database_path=".cache_langchain.db"))

llm_generator = ChatOpenRouter(model_name=LLM_NAME, cache=False,
                               temperature=TEMPERATURE) 

In [4]:
from rich import print

training_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/financial"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')
"""
print(training_code)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/financial"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

# Baseline agent

In [ ]:
from caia.benchmark.baseline import StandardGraph
standard_graph = StandardGraph(llm_generator, debug=False)

initial_state = {
    "model_code": training_code,
    "metrics": {
        "model_old_score": {
            "on_new_data": 0.7166666666666667,
            "on_old_data": 0.9133333333333333
        }
    },
    "max_iterations": MAX_ITERATIONS,
    "dataset_description": dataset_description
}

output = standard_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/baseline_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

Max iterations set to: 1

🚀 Starting Improved Baseline Model Improvement Process

Dataset: Loan Default Prediction Data

Features: 10 total, 7 numerical, 3 categorical

Executing Node: improve_code

...

Initial metrics: {'old_distribution': 0.9133333333333333, 'new_distribution': 0.7166666666666667}

Final metrics: {}

Improvement path: 0 entries

Total tokens used: 1774

Results saved to: results/baseline_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_71441fa6.yaml

# ReAct agent

In [6]:
from caia.benchmark.react import ReactImprover

# Initialize the React improver with your LLM
react_graph = ReactImprover(llm_generator)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": {
        "model_old_score": {
            "on_new_data": 0.7166666666666667,
            "on_old_data": 0.9133333333333333
        }
    },
    "max_iterations": MAX_ITERATIONS,
    "dataset_description": dataset_description
}

# Run the agent
output = react_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/react_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

Max iterations set to: 1

🚀 Starting React-based Model Improvement Process

Dataset: Loan Default Prediction Data

Error handling: stopping after 3 consecutive failures

Features: 10 total, 7 numerical, 3 categorical

...

Completion: 16213

Total: 45166

Error in graph execution: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit
by setting the `recursion_limit` config key.

Traceback (most recent call last):
  File "/home/guess/phd/improver/caia/benchmark/react.py", line 1158, in run
    for output in self.decision_procedure.stream(typed_state):
  File "/home/guess/mambaforge/envs/dspy/lib/python3.11/site-packages/langgraph/pregel/__init__.py", line 1000, in stream
    raise GraphRecursionError(
langgraph.errors.GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.


GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

# Plan and execute agent

In [7]:
from caia.benchmark.plan_and_execute import PlanAndExecuteGraph

# Initialize with max_failures parameter
plan_execute_graph = PlanAndExecuteGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=3  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": {
        "model_old_score": {
            "on_new_data": 0.7166666666666667,
            "on_old_data": 0.9133333333333333
        }
    },
    "dataset_description": dataset_description
}

# Run the agent
output = plan_execute_graph.run(initial_state)

# create a short version of uuid using python


short_uuid = str(uuid.uuid4())[:8]
filename = f"results/planexecute_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"


save_yaml_results(output, filename)

🚀 Starting Plan-and-Execute Model Improvement Process

Dataset: Loan Default Prediction Data

Error handling: stopping after 3 consecutive failures

Features: 10 total, 7 numerical, 3 categorical

🧠 PLANNING STEP

...

Initial metrics: {'old_distribution': 0.9133333333333333, 'new_distribution': 0.7166666666666667}

Final metrics: {}

Improvement path: 0 entries

Total tokens used: 15485

Results saved to: results/planexecute_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_3730c043.yaml

# Reflection agent

In [8]:
from caia.benchmark.reflection import ReflectionGraph


# Initialize with both max_iterations and max_failures parameters
reflection_graph = ReflectionGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=3  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": {
        "model_old_score": {
            "on_new_data": 0.7166666666666667,
            "on_old_data": 0.9133333333333333
        }
    },
    "dataset_description": dataset_description
}

# Run the agent
output = reflection_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/reflection_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

🚀 Starting Reflection-Based Model Improvement Process

Dataset: Loan Default Prediction Data

Error handling: stopping after 3 consecutive failures

Features: 10 total, 7 numerical, 3 categorical

🔍 GENERATING IMPROVED CODE

...

Final metrics: {'old_distribution': 0.805, 'new_distribution': 0.7333333333333333}

Improvement path: 1 entries

Reflections: 1

Total tokens used: 5846

Results saved to: results/reflection_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_79c3d579.yaml

# ToT

In [44]:
from caia.benchmark.tot import TreeOfThoughtsGraph
import json



# Initialize with ToT-specific parameters
tot_graph = TreeOfThoughtsGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    beam_width=3,           # Number of candidates to keep after pruning
    num_candidates=3,       # Number of candidates to generate in each expansion
    threshold=0.9,          # Score threshold for accepting a solution
    max_depth=3,            # Maximum search depth in the tree
    max_failures=3          # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": {
        "model_old_score": {
            "on_new_data": 0.7166666666666667,
            "on_old_data": 0.9133333333333333
        }
    },
    "dataset_description": dataset_description
}

# Run the agent
output = tot_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/tot_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

🚀 Starting Tree of Thoughts Model Improvement Process

Parameters: max_iterations=1, beam_width=3, num_candidates=3

Error handling: stopping after 3 consecutive failures

Dataset: Loan Default Prediction Data

Features: 10 total, 7 numerical, 3 categorical

...

Final metrics: {'old_distribution': 0.915, 'new_distribution': 0.8666666666666667}

Improvement path: 1 entries

Total tokens used: 2729

Execution stats: 1 successes, 0 failures

Results saved to: results/tot_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_9e60f07a.yaml

# Self-Discovery

In [10]:
from caia.benchmark.self_discover import SelfDiscoverGraph

# Initialize with both max_iterations and max_failures
self_discovery_agent = SelfDiscoverGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=4  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": {
        "model_old_score": {
            "on_new_data": 0.7167,
            "on_old_data": 0.9133
        }
    },
    "dataset_description": dataset_description
}

# Run the agent
output = self_discovery_agent.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/selfdiscovery_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)


🚀 Starting Self-Discovery Model Improvement Process
Dataset: Loan Default Prediction Data
Error handling: stopping after 4 consecutive failures
Features: 10 total, 7 numerical, 3 categorical

🔍 SELECTING REASONING MODULES
Selected modules: 3
- 1. How could I simpl...
- 2. What are the key ...
- 3. How can I impleme...

Current Token Usage:
Prompt: 0
Completion: 0
Total: 0

🛠️ ADAPTING MODULES
Adapted modules: **Simplified Problem**

To simplify the problem, let's focus on a binary classification task: predicting the likelihood of a consumer defaulting on a loan based on their financial data. We'll use the existing datasets as is, without any modifications.

**Key Techniques for Improving Distribution Shifts**

To improve the model's robustness to distribution shifts, we'll employ the following techniques:

1.  **Data Ensembling**: Combine the old and new datasets to create a more comprehensive training set, reducing the impact of new data mismatch.
2.  **Regularization**: Use L1 and L

Results saved to: 
results/selfdiscovery_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_aa820f4d.yaml

# Improver

## Semantic memory

### Training code

In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/financial"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

Model trained and evaluated on the old distribution: 0.9133333333333333

In [33]:
from caia.memory import WorkingMemory, EpisodicMemory, SemanticMemory
from caia.memory import Dataset


# tools = get_tools([calculate_trust_score])


# At the beginning, the agent has 1 entry in the semantic memory. 
# Here we put the path of each dataset file in the semantic memory.
dataset_old = Dataset(X_train=f"{dataset_folder}/X_train_old.csv",
                                     X_test=f"{dataset_folder}/X_test_old.csv",
                                     y_train=f"{dataset_folder}/y_train_old.csv",
                                     y_test=f"{dataset_folder}/y_test_old.csv",
                                     description=dataset_description)

model_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/financial"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')
"""

init_semantic_memory = SemanticMemory(dataset_old=dataset_old, 
                                        model_object=model_old, 
                                        model_code=model_code)
# semantic_memory
print(init_semantic_memory.model_code)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/financial"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

## Episodic memory

In [34]:
from caia.memory import Dataset
from docarray import DocList

dataset_new = Dataset(X_train=f"{dataset_folder}/X_train_new.csv",
                        X_test=f"{dataset_folder}/X_test_new.csv",
                        y_train=f"{dataset_folder}/y_train_new.csv",
                        y_test=f"{dataset_folder}/y_test_new.csv",
                        description=dataset_description)


first_episodic_memory = EpisodicMemory(dataset_new=dataset_new,
                                        quick_insight={},
                                       deep_insight=None)
init_episodic_memory = DocList[EpisodicMemory]([first_episodic_memory])
init_episodic_memory[0]


📄 EpisodicMemory : 803afb7 ...
╭────────────────────────────┬─────────╮
│ Attribute                  │ Value   │
├────────────────────────────┼─────────┤
│ quick_insight: dict        │ {}      │
╰────────────────────────────┴─────────╯
└── 🔶 dataset_new: Dataset
    └── 📄 Dataset : 0a420cd ...
        ╭───────────────────┬───────────────────────────────────────────────────────────────────────╮
        │ Attribute         │ Value                                                                 │
        ├───────────────────┼───────────────────────────────────────────────────────────────────────┤
        │ X_train: str      │ datasets/financial/X_train_new.csv                                    │
        │ X_test: str       │ datasets/financial/X_test_new.csv                                     │
        │ y_train: str      │ datasets/financial/y_train_new.csv                                    │
        │ y_test: str       │ datasets/financial/y_test_new.csv                                     │
        │ description: dict │ {'NUM_SAMPLES': 2000, 'FEATURES': ['Age', 'Income' ... } (length: 11) │
        ╰───────────────────┴───────────────────────────────────────────────────────────────────────╯

## Fast graph

In [36]:
from caia.fast.fast_graph import FastGraph
from caia.utils import save_yaml_results

working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph={},
    improvement_history=[],
)

fast_graph = FastGraph(llm_generator, debug=False)
output_fast_graph = fast_graph.run(working_memory)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/fast_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_fast_graph, filename)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: generate_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

No slow graph insights available, using basic retraining approach

╭─────────────────────────────────────────────── new_training_code ───────────────────────────────────────────────╮
│ new_training_code: |                                                                                            │
│     import yaml                                                                                                 │
│     from sklearn.ensemble import RandomForestClassifier                                                         │
│     from sklearn.metrics import accuracy_score                                                                  │
│     import pandas as pd                                                                                         │
│                                                                                                                 │
│     # Initialize metrics dictionaries                                                                           │
│     model_new_score = {                                                                                         │
│         'on_new_data': 0.0,                                                                                     │
│         'on_old_data': 0.0                                                                                      │
│     }                                                                                                           │
│     model_old_score = {                                                                                         │
│         'on_new_data': 0.0,                                                                                     │
│         'on_old_data': 0.0                                                                                      │
│     }                                                                                                           │
│                                                                                                                 │
│     # model architecture and parameters                                                                         │
│     model = RandomForestClassifier(random_state=42)                                                             │
│                                                                                                                 │
│     # load the old data                                                                                         │
│     dataset_folder = "datasets/financial"                                                                       │
│     X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                              │
│     X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                │
│     y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                           │
│     y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                             │
│                                                                                                                 │
│     # Train and evaluate old model                                                                              │
│     model_old = model                                                                                           │
│     model_old.fit(X_train_old, y_train_old)                                                                     │
│                                                                                                                 │
│     # Test old model on old test set                                                                            │
│     old_accuracy_old = accuracy_score(y_test_old, model_old.predict(X_test_old))                                │
│     print(f'Old model trained and evaluated on the old distribution: {old_accuracy_old}')                       │
│     model_old_score['on_old_data'] = float(old_accurac

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


...

╭─────────────────────────────────────────────── execution_success ───────────────────────────────────────────────╮
│ True                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── extracted_metrics ───────────────────────────────────────────────╮
│ {'model_old_score': {'on_old_data': 0.9133333333333333, 'on_new_data': 0.7166666666666667}, 'model_new_score':  │
│ {'on_old_data': 0.9066666666666666, 'on_new_data': 0.8}}                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── iteration_count ────────────────────────────────────────────────╮
│ 1                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.0833                                                                                      │
│   Old Distribution: -0.0067                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Results saved to: results/fast_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_ab9e9317.yaml

## Slow graph

In [37]:
from caia.slow.slow_graph import SlowGraph
from caia.utils import save_yaml_results

slow_graph = SlowGraph(llm_generator, debug=False)
working_memory["max_iterations"] = MAX_ITERATIONS
working_memory["max_failures"] = 5
output_slow_graph = slow_graph.run(working_memory)


short_uuid = str(uuid.uuid4())[:8]
filename = f"results/slow_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_slow_graph, filename)

Max iterations set to: 1

Max consecutive failures set to: 5

==================== STARTING ITERATION 1/1 ====================

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: check_fast_graph_results                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Detected Fast Graph Results from generations_fast_graph: --------------------------------------------------

...

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Strategy: hyperparameter_tuning                                                                                 │
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.1667                                                                                      │
│   Old Distribution: -0.0033                                                                                     │
│ Evaluation: reject                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Strategy Progress ───────────────────────────────────────────────╮
│   [○] model_selection                                                                                           │
│ → [✓] hyperparameter_tuning                                                                                     │
│   [✓] ensemble_method                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Iteration 1 time: 166.93 seconds

Terminating after iteration 1 due to convergence or no improvement

Results saved to: results/slow_temp_1.0_max_iter_1_llm_llama-3.1-8b-instruct_dataset_financial_66c3c7f1.yaml

## Fast graph again

In [43]:
from caia.fast.fast_graph import FastGraph
from caia.utils import save_yaml_results

working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph=output_slow_graph,
    improvement_history=[],
)

fast_graph = FastGraph(llm_generator, debug=False)
output_fast_graph = fast_graph.run(working_memory)

short_uuid = str(uuid.uuid4())[:8]
print(short_uuid)
filename = f"results/improver_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_fast_graph, filename)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: generate_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Using insights from slow graph to enhance retraining code generation

╭─────────────────────────────────────────────── new_training_code ───────────────────────────────────────────────╮
│ new_training_code: |                                                                                            │
│     import yaml                                                                                                 │
│     import pandas as pd                                                                                         │
│     from sklearn.ensemble import RandomForestClassifier                                                         │
│     from sklearn.metrics import accuracy_score                                                                  │
│     from sklearn.model_selection import train_test_split                                                        │
│     from sklearn.utils import class_weight                                                                      │
│     from sklearn.exceptions import ConvergenceWarning                                                           │
│     import warnings                                                                                             │
│     warnings.filterwarnings("ignore", category=ConvergenceWarning)                                              │
│                                                                                                                 │
│     # Load data from specified folder                                                                           │
│     dataset_folder = "datasets/financial"                                                                       │
│     X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                              │
│     X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                │
│     y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                           │
│     y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                             │
│                                                                                                                 │
│     # Import necessary data for new model                                                                       │
│     X_train_new = pd.read_csv(f"{dataset_folder}/X_train_new.csv")                                              │
│     y_train_new = pd.read_csv(f"{dataset_folder}/y_train_new.csv").squeeze("columns")                           │
│     X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")                                                │
│     y_test_new = pd.read_csv(f"{dataset_folder}/y_test_new.csv").squeeze("columns")                             │
│                                                                                                                 │
│     # Evaluate the improved model on old tests set                                                              │
│     improved_model = RandomForestClassifier(                                                                    │
│         n_estimators=200,                                                                                       │
│         max_depth=10,                                                                                           │
│         min_samples_split=5,                                                                                    │
│         min_samples_leaf=1,                                                                                     │
│         max_features=7,                                                                                         │
│         random_state=42                                                                                         │
│     )                                                                                                           │
│                                                       

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


...

╭──────────────────────────────────────────────── iteration_count ────────────────────────────────────────────────╮
│ 1                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                            Node: fix_retraining_code                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── fixed_code ───────────────────────────────────────────────────╮
│ fixed_code: |                                                                                                   │
│   import yaml                                                                                                   │
│   import pandas as pd                                                                                           │
│   from sklearn.ensemble import RandomForestClassifier                                                           │
│   from sklearn.metrics import accuracy_score                                                                    │
│   from sklearn.model_selection import train_test_split                                                          │
│   from sklearn.utils import class_weight                                                                        │
│   from sklearn.exceptions import ConvergenceWarning                                                             │
│   import warnings                                                                                               │
│   warnings.filterwarnings("ignore", category=ConvergenceWarning)                                                │
│                                                                                                                 │
│   # Load data from specified folder                                                                             │
│   dataset_folder = "datasets/financial"                                                                         │
│   X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                                │
│   X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                  │
│   y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                             │
│   y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                               │
│                                                                                                                 │
│   # Import necessary data for new model                                                                         │
│   X_train_new = pd.read_csv(f"{dataset_folder}/X_train_new.csv")                                                │
│   y_train_new = pd.read_csv(f"{dataset_folder}/y_train_new.csv").squeeze("columns")                             │
│   X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")                                                  │
│   y_test_new = pd.read_csv(f"{dataset_folder}/y_test_new.csv").squeeze("columns")                               │
│                                                                                                                 │
│   # Evaluate the improved model on old tests set                                                                │
│   improved_model = RandomForestClassifier(                                                                      │
│       n_estimators=200,                                                                                         │
│       max_depth=10,                                                                                             │
│       min_samples_split=5,                                                                                      │
│       min_samples_leaf=1,                                                                                       │
│       max_features=7,                                                                                           │
│       random_state=42                                                                                           │
│   )                                                                                                             │
│                                                       

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

KeyError: 'new_training_code'

## Improver

In [18]:
working_memory

{'episodic_memory': <DocList[EpisodicMemory] (length=1)>,
 'semantic_memory': SemanticMemory(id='d20de7e820a8fc3701975092a486899b', dataset_old=Dataset(id='9a6f8eb95b5c75e63801f0b45a6b8ea7', X_train='datasets/financial/X_train_old.csv', X_test='datasets/financial/X_test_old.csv', y_train='datasets/financial/y_train_old.csv', y_test='datasets/financial/y_test_old.csv', description={'NUM_SAMPLES': 2000, 'FEATURES': ['Age', 'Income', 'Credit Score', 'Loan Amount', 'Loan Term', 'Interest Rate', 'Employment Length', 'Home Ownership', 'Marital Status', 'Dependents'], 'NUMERICAL_FEATURES': ['Age', 'Income', 'Credit Score', 'Loan Amount', 'Loan Term', 'Interest Rate', 'Employment Length'], 'CATEGORICAL_FEATURES': ['Home Ownership', 'Marital Status', 'Dependents'], 'LABEL': 'Loan Default', 'COLUMN_TYPES': {'Age': 'int', 'Income': 'float', 'Credit Score': 'int', 'Loan Amount': 'float', 'Loan Term': 'int', 'Interest Rate': 'float', 'Employment Length': 'int', 'Home Ownership': 'int', 'Marital Sta

In [ ]:
from caia.improver import Improver
from caia.memory import WorkingMemory
from caia.utils import save_yaml_results
import uuid

# Initialize working memory
working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph={},
    improvement_history=[],
)

# Create the improver agent
improver = Improver(
    llm_generator, 
    max_iterations=MAX_ITERATIONS, 
    max_failures=5,
    debug=False
)

# Run the agent
output = improver.run(working_memory)

# Create standardized filename
short_uuid = str(uuid.uuid4())[:8]
filename = f"results/improver_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"

# Save results
save_yaml_results(output, filename)

In [ ]:
# Run the agent
output = improver.run(working_memory)

# Create standardized filename
short_uuid = str(uuid.uuid4())[:8]
filename = f"results/improver_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"

# Save results
save_yaml_results(output, filename)

# AutoML

In [18]:
from caia.benchmark.automl import run_automl_benchmark
import yaml


# Prepare initial metrics
initial_metrics = {
    "model_old_score": {
        "on_new_data": 0.7166666666666667,
        "on_old_data": 0.9133333333333333
    }
}

# Run AutoML benchmark
result = run_automl_benchmark(
    training_code, 
    dataset_description, 
    initial_metrics,
    max_iterations=3
)

# Save results
import uuid
short_uuid = str(uuid.uuid4())[:8]
filename = f"results/automl_max_iter_{MAX_ITERATIONS}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
with open(filename, 'w') as f:
    yaml.dump(result, f)


AutoML initialized with 7 numerical features and 3 categorical features
Max iterations: 3
Base model - Old distribution score: 0.9133
Base model - New distribution score: 0.7167


==================== ITERATION 1/3 ====================


Trying random_forest...
  random_forest - Old distribution: 0.9083, New distribution: 0.8833
  Best params: {'classifier__max_depth': 10, 'classifier__n_estimators': 100, 'classifier__random_state': 42}

Trying gradient_boosting...
  gradient_boosting - Old distribution: 0.9117, New distribution: 0.8333
  Best params: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__random_state': 42}

Trying logistic_regression...
  logistic_regression - Old distribution: 0.8600, New distribution: 0.8333
  Best params: {'classifier__C': 0.1, 'classifier__random_state': 42}

Iteration 1 complete in 6.18 seconds
Best model: random_forest
Old distribution: 0.9083, New distribution: 0.8833


==================== 

/home/guess/mambaforge/envs/dspy/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/guess/mambaforge/envs/dspy/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/guess/mambaforge/envs/dspy/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/guess/mambaforge/envs/dspy/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use t

  adaboost - Old distribution: 0.8500, New distribution: 0.8333
  Best params: {'classifier__learning_rate': 0.5, 'classifier__n_estimators': 100, 'classifier__random_state': 42}

Iteration 2 complete in 23.05 seconds
Best model: random_forest
Old distribution: 0.9083, New distribution: 0.8833


==================== ITERATION 3/3 ====================


Trying gradient_boosting...
  gradient_boosting - Old distribution: 0.9100, New distribution: 0.8500
  Best params: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 7, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 500, 'classifier__random_state': 42, 'classifier__subsample': 0.6}

Trying svm...
  svm - Old distribution: 0.9317, New distribution: 0.8667
  Best params: {'classifier__C': 10.0, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf', 'classifier__probability': True, 'classifier__random_state': 42}

Iteration 3 complete in 152.32 seconds
Best model: svm
Old distribution: 0.9317, New distributio